In [1]:
import pandas as pd
import numpy as np
import os

import ast

from tqdm.notebook import tqdm

import matplotlib.pyplot as plt

In [2]:
# Path to folder with JSON files
folder_path = '/content/drive/MyDrive/datasets/wines'

progress_bar = tqdm(os.listdir(folder_path), desc="Processing files", leave=True)

# Initialize an empty list to store DataFrames
dataframes = []

# Go through the files in the folder and load them into the DataFrame
for file in os.listdir(folder_path):
    if file.endswith('.json'):
        file_path = os.path.join(folder_path, file)
        try:
            df = pd.read_json(file_path)
            dataframes.append(df)
        except Exception as e:
            print(f"File loading error {file}: {e}")

# Combine all DataFrames into one
merged_dataframe = pd.concat(dataframes, ignore_index=True)

# Save to CSV
csv_filename = '/content/drive/MyDrive/datasets/merged_wine_data.csv'
merged_dataframe.to_csv(csv_filename, index=False)

print(f"Successfully merged and saved in {csv_filename}")

Processing files:   0%|          | 0/1073 [00:00<?, ?it/s]

Successfully merged and saved in /content/drive/MyDrive/datasets/merged_wine_data.csv


In [3]:
dataframe = pd.read_csv('/content/drive/MyDrive/datasets/merged_wine_data.csv')

In [4]:
dataframe.head

<bound method NDFrame.head of                                                    wines
0      {'name': 'Moscato', 'price': {'id': 33340883, ...
1      {'name': 'Indisciplinato', 'price': {'id': 309...
2      {'name': 'Reserve Collection Pinot Noir', 'pri...
3      {'name': "Stokes' Ghost Petite Sirah", 'price'...
4      {'name': 'Défier Coquelicot Vineyard Merlot', ...
...                                                  ...
26820  {'name': 'Bloom & Bramble Chardonnay', 'price'...
26821  {'name': 'Royal Seyssel Grand Cru Régional', '...
26822  {'name': 'Grande Reserve Côtes de Bourg Rouge'...
26823  {'name': 'Argenina Gran Selezione Chianti Clas...
26824  {'name': 'Private Reserve Pinot Noir', 'price'...

[26825 rows x 1 columns]>

In [25]:
dataframe.iloc[11]['wines']

"{'name': 'Merlot', 'price': {'id': 28440603, 'merchant_id': 31839, 'amount': 29.99, 'discounted_from': None, 'discount_percent': None, 'type': 'vc', 'sku': '63574', 'url': 'https://www.classycorksdc.com/wines/Starmont-Winery-Merlot-2018-w4314924v2?utm_source=Vivino&utm_medium=Feed', 'visibility': 1, 'bottle_type_id': 1, 'currency': {'code': 'USD', 'name': 'US Dollars', 'prefix': '$', 'suffix': None}, 'bottle_type': {'id': 1, 'name': 'Bottle (0.75l)', 'short_name': 'bottle', 'short_name_plural': 'bottles', 'volume_ml': 750}}, 'taste': {'structure': {'acidity': 1.857961, 'fizziness': None, 'intensity': 3.9818263, 'sweetness': 1.9654433, 'tannin': 2.1349912, 'user_structure_count': 16, 'calculated_structure_count': 121}, 'flavor': [{'group': 'oak', 'stats': {'count': 79, 'score': 12550, 'mentions_count': 139}, 'primary_keywords': [{'id': 292, 'name': 'oak', 'count': 39}, {'id': 434, 'name': 'vanilla', 'count': 38}, {'id': 101, 'name': 'chocolate', 'count': 16}, {'id': 422, 'name': 'tobac

In [19]:
data_list = []

for i in range(dataframe.shape[0]):
    data_dict = ast.literal_eval(dataframe['wines'][i])
    selected_data = {}
    selected_data['style_name'] = data_dict['name']
    selected_data['price'] = data_dict['price']['amount']
    selected_data['currency'] = data_dict['price']['currency']['code']
    selected_data['volume'] = data_dict['price']['bottle_type']['volume_ml']

    if data_dict['taste'] is not None and data_dict['taste']['structure'] is not None:
        selected_data['acidity'] = data_dict['taste']['structure'].get('acidity', None)
        selected_data['fizziness'] = data_dict['taste']['structure'].get('fizziness', None)
        selected_data['sweetness'] = data_dict['taste']['structure'].get('sweetness', None)
        selected_data['tannin'] = data_dict['taste']['structure'].get('tannin', None)
        selected_data['user_structure_count'] = data_dict['taste']['structure'].get('user_structure_count', None)
        selected_data['calculated_structure_count'] = data_dict['taste']['structure'].get('calculated_structure_count', None)
        # selected_data['flavour_list'] = data_dict['taste']['flavor']


    reviews = data_dict['reviews']
    vintages = [reviews[i]['vintage'] for i in range(len(reviews))]
    selected_data['wine_name'] = vintages[0]['name']
    region = vintages[0]['wine']['region']
    selected_data['rating'] = reviews[0]['rating']
    selected_data['region_name'] = region['name']
    selected_data['country'] = region['country']['name']
    selected_data['winery'] = vintages[0]['wine']['winery']['name']
    selected_data['image'] = vintages[0]['image']['location']

    selected_data['flavours'] = []
    for review in reviews:
        flavor_matches = review.get('flavor_word_matches')
        if flavor_matches is not None:
            selected_data['flavours'].extend(match.get('match') for match in flavor_matches)



    data_list.append(selected_data)

df = pd.DataFrame(data_list)


In [20]:
df.head()

,style_name,price,currency,volume,acidity,fizziness,sweetness,tannin,user_structure_count,calculated_structure_count,wine_name,rating,region_name,country,winery,image,flavours
0,Moscato,12.990000,USD,750,3.067054,NaN,3.271635,NaN,18.0,192.0,Seven Daughters Moscato U.V.,4.1,Veneto,Italy,Seven Daughters,//images.vivino.com/thumbs/mE6S9dUrSUOFI8qTodT...,"[lychee, minerals, white peach, apple, pear]"
1,Indisciplinato,27.990000,USD,750,3.442514,NaN,1.623086,NaN,9.0,1.0,Tenuta San Marcello Indisciplinato U.V.,2.5,Marche,Italy,Tenuta San Marcello,//images.vivino.com/labels/kJVZn2JsRiiUc7Xfz7x...,"[dried mango, orange]"
2,Reserve Collection Pinot Noir,24.990000,USD,750,3.870318,NaN,1.416349,1.883968,1.0,3.0,Erath Reserve Collection Pinot Noir 2019,4.2,Willamette Valley,United States,Erath,//images.vivino.com/labels/DU6Kr8eeTGSC9Cl-5MV...,[]
3,Stokes' Ghost Petite Sirah,29.916667,USD,750,3.413706,NaN,1.921906,4.333656,22.0,145.0,Scheid Vineyards Stokes' Ghost Petite Sirah U.V.,4.1,Monterey,United States,Scheid Vineyards,//images.vivino.com/thumbs/VhNssH95RtKFPe73dxK...,[]
4,Défier Coquelicot Vineyard Merlot,29.990000,USD,750,1.827581,NaN,1.946129,2.162903,2.0,11.0,J.Brix Défier Coquelicot Vineyard Merlot U.V.,4.1,Santa Ynez Valley,United States,J.Brix,//images.vivino.com/thumbs/DWwwq3AtSdWLB_SrbxY...,"[black fruit, black olive, plum]"


In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26825 entries, 0 to 26824
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   style_name                  26825 non-null  object 
 1   price                       26825 non-null  float64
 2   currency                    26825 non-null  object 
 3   volume                      26825 non-null  int64  
 4   acidity                     24937 non-null  float64
 5   fizziness                   451 non-null    float64
 6   sweetness                   24486 non-null  float64
 7   tannin                      14379 non-null  float64
 8   user_structure_count        24937 non-null  float64
 9   calculated_structure_count  24937 non-null  float64
 10  wine_name                   26825 non-null  object 
 11  rating                      26825 non-null  float64
 12  region_name                 26825 non-null  object 
 13  country                     268

In [27]:
df.describe()

,price,volume,acidity,fizziness,sweetness,tannin,user_structure_count,calculated_structure_count,rating
count,26825.000000,26825.000000,24937.000000,451.000000,24486.000000,14379.000000,24937.000000,24937.000000,26825.000000
mean,24.146525,747.260019,3.404173,3.030112,1.864479,3.093038,62.669046,125.370333,4.086837
std,4.557660,31.974264,0.639423,1.053950,0.754787,0.600516,284.137408,412.053915,0.451426
min,4.990000,375.000000,1.378824,1.000000,1.000000,1.368427,0.000000,0.000000,1.000000
25%,19.990000,750.000000,3.000000,3.500000,1.422000,2.801188,2.000000,6.000000,4.000000
50%,24.990000,750.000000,3.381946,3.500000,1.662057,3.193358,10.000000,30.000000,4.000000
75%,27.990000,750.000000,3.908907,3.500000,2.021250,3.407533,41.000000,93.000000,4.400000
max,30.000000,1000.000000,5.000000,4.965238,5.000000,4.691349,10940.000000,19037.000000,5.000000


In [28]:
df.to_csv('/content/drive/MyDrive/datasets/wines_processed/wines.csv')